In [23]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
# actual
actual_solar_raleigh = pd.read_csv('Raleigh/6ef6c64fbb3bfd36bcb328cc956df78d/Actual_35.85_-78.65_2006_DPV_37MW_5_Min.csv')

actual_solar_raleigh['LocalTime'] = pd.to_datetime(actual_solar_raleigh['LocalTime'])
actual_solar_raleigh["TIME"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.time
actual_solar_raleigh['Minute'] = pd.to_datetime(actual_solar_raleigh['TIME'],format='%H:%M:%S').dt.minute
# remove other times, keep only 00 minutes
actual_solar_raleigh = actual_solar_raleigh[actual_solar_raleigh['Minute'] == 0]
actual_solar_raleigh['Power(MW)_actual'] = actual_solar_raleigh['Power(MW)']

In [40]:
ha4_solar_raleigh = pd.read_csv('Raleigh/6ef6c64fbb3bfd36bcb328cc956df78d/HA4_35.85_-78.65_2006_DPV_37MW_60_Min.csv')

ha4_solar_raleigh['LocalTime'] = pd.to_datetime(ha4_solar_raleigh['LocalTime'])
ha4_solar_raleigh['Power(MW)_ha4'] = ha4_solar_raleigh['Power(MW)']

In [41]:
df2 = pd.merge(ha4_solar_raleigh, actual_solar_raleigh, how='inner', on='LocalTime')
df2 = df2[(df2['Power(MW)_actual'] > 0) | (df2['Power(MW)_ha4'] > 0)]

ha4 = df2['Power(MW)_ha4']

actual = df2['Power(MW)_actual']

In [42]:
R2_Score_dtr = round(r2_score(ha4,actual) * 100, 2)
print("R2 Score : ",R2_Score_dtr,"%")


R2 Score :  75.45 %


### The 4 hour forcast data that is provided with the actual data has a R2 score of 75.45%

# Now check if we can use Charlotte data to predict Raleigh power output.

In [75]:
# actual
actual_solar_raleigh = pd.read_csv('Raleigh/6ef6c64fbb3bfd36bcb328cc956df78d/Actual_35.85_-78.65_2006_DPV_37MW_5_Min.csv')

actual_solar_raleigh['LocalTime'] = pd.to_datetime(actual_solar_raleigh['LocalTime'])
# adding separate time and date columns
actual_solar_raleigh["Year"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.year
actual_solar_raleigh['Day'] = pd.to_datetime(actual_solar_raleigh['LocalTime']).dt.day
actual_solar_raleigh['Month'] = pd.to_datetime(actual_solar_raleigh['LocalTime']).dt.month
actual_solar_raleigh["TIME"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.time
actual_solar_raleigh["DATE"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.date
# add hours and minutes for ml models
actual_solar_raleigh['Hour'] = pd.to_datetime(actual_solar_raleigh['TIME'],format='%H:%M:%S').dt.hour
actual_solar_raleigh['Minute'] = pd.to_datetime(actual_solar_raleigh['TIME'],format='%H:%M:%S').dt.minute
actual_solar_raleigh['TOTAL MINUTES PASS'] = actual_solar_raleigh['Minute'] + actual_solar_raleigh['Hour']*60
actual_solar_raleigh = actual_solar_raleigh[(actual_solar_raleigh['Minute'] == 0) | (actual_solar_raleigh['Minute'] == 30)]
actual_solar_raleigh.columns

Index(['LocalTime', 'Power(MW)', 'Year', 'Day', 'Month', 'TIME', 'DATE',
       'Hour', 'Minute', 'TOTAL MINUTES PASS'],
      dtype='object')

In [76]:
# Charlotte Irradiance data
irradiance_data_charlotte = pd.read_csv('Charlotte/653351892e546d5d55892a5e2a054320/Modified_1045140_35.21_-80.86_2006.csv')
irradiance_data_charlotte['LocalTime'] = pd.to_datetime(irradiance_data_charlotte[['Year', 'Month', 'Day', 'Hour', 'Minute']])
irradiance_data_charlotte["TIME"] = pd.to_datetime(irradiance_data_charlotte["LocalTime"]).dt.time
irradiance_data_charlotte["DATE"] = pd.to_datetime(irradiance_data_charlotte["LocalTime"]).dt.date
irradiance_data_charlotte.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'DHI', 'DNI', 'Dew Point',
       'Surface Albedo', 'Wind Speed', 'Relative Humidity', 'Temperature',
       'Pressure', 'Clearsky DNI', 'Clearsky DHI', 'Clearsky GHI', 'GHI',
       'Solar Zenith Angle', 'Cloud Type', 'Fill Flag', 'Wind Direction',
       'Precipitable Water', 'Global Horizontal UV Irradiance (280-400nm)',
       'Global Horizontal UV Irradiance (295-385nm)', 'LocalTime', 'TIME',
       'DATE'],
      dtype='object')

In [81]:
# evaluate an xgboost regression model on the housing dataset
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor


def learn(clt, solar, w, s):
    df2 = pd.merge(clt, solar, how='inner', on='LocalTime')
    df2 = df2[(df2['Power(MW)'] > 0) | (df2['DHI'] > 0) | (df2['DNI'] > 0) | (df2['GHI'] > 0)]

    X = df2[['DHI', 'DNI', 'Dew Point',
       'Surface Albedo', 'Wind Speed', 'Relative Humidity', 'Temperature',
       'Pressure', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'GHI',
       'Solar Zenith Angle', 'Cloud Type', 'Fill Flag', 'Wind Direction',
       'Precipitable Water', 'Global Horizontal UV Irradiance (280-400nm)',
       'Global Horizontal UV Irradiance (295-385nm)']]
    y = df2['Power(MW)']
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=69)
    
    model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    scores = absolute(scores)
    print("******************************************")
    print('Weather shift: ', w)
    print('Solar shift: ', s)
    print("******************")
    print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )
    
    model.fit(X_train,y_train)

    score_lr = 100*model.score(X_test,y_test)
    print(f'XGBR Model score = {score_lr:4.4f}%')

    y_pred_lr = model.predict(X_test)
    R2_Score_lr = round(r2_score(y_pred_lr,y_test) * 100, 2)

    print("R2 Score : ",R2_Score_lr,"%")
    print("******************************************")


In [82]:
weather_shifts = [0, 2, 4, 6, 8] # shifting weather readings by 0, 1, 2, 3, or 4 hours
sunlight_shifts = [0, 1] # shifting daylight readings by 0 or 30 mins

In [83]:
for w in weather_shifts:
    for s in sunlight_shifts:
        
        dup_clt_data = irradiance_data_charlotte.copy()
        
        weather_data = dup_clt_data[['Dew Point',
           'Surface Albedo', 'Wind Speed', 'Relative Humidity', 'Temperature',
           'Pressure', 'Cloud Type', 'Fill Flag', 'Wind Direction',
           'Precipitable Water']]
        sunlight_data = dup_clt_data[['DHI', 'DNI', 'Clearsky DNI', 'Clearsky DHI', 'Clearsky GHI', 'GHI',
           'Solar Zenith Angle', 'Global Horizontal UV Irradiance (280-400nm)',
           'Global Horizontal UV Irradiance (295-385nm)']]
        time_data = dup_clt_data[['Year', 'Month', 'Day', 'Hour', 'Minute','LocalTime', 'TIME',
           'DATE']]
        
        # shift weather and solar data and combine back
        weather_data = weather_data.shift(periods=w)
        sunlight_data = sunlight_data.shift(periods=s)
        time_data = time_data.join(weather_data)
        temp = time_data.join(sunlight_data)
        
        temp_clt = temp[(temp['Minute'] == 0) | (temp['Minute'] == 30)]
        
        learn(temp_clt, actual_solar_raleigh, w, s)
        

******************************************
Weather shift:  0
Solar shift:  0
******************
Mean MAE: 1.835 (0.064)
XGBR Model score = 89.8880%
R2 Score :  88.59 %
******************************************
******************************************
Weather shift:  0
Solar shift:  1
******************
Mean MAE: 2.262 (0.055)
XGBR Model score = 86.1959%
R2 Score :  83.65 %
******************************************
******************************************
Weather shift:  2
Solar shift:  0
******************
Mean MAE: 1.816 (0.069)
XGBR Model score = 89.9148%
R2 Score :  88.54 %
******************************************
******************************************
Weather shift:  2
Solar shift:  1
******************
Mean MAE: 2.042 (0.066)
XGBR Model score = 87.5859%
R2 Score :  85.33 %
******************************************
******************************************
Weather shift:  4
Solar shift:  0
******************
Mean MAE: 1.818 (0.057)
XGBR Model score = 89.2263%
R2 Score

# Grab a copy of the original Charlotte data and and shift the weather_data part up by 4 hours and save it

In [93]:
# Save 
dup_clt_data = irradiance_data_charlotte.copy()

weather_data = dup_clt_data[['Dew Point',
   'Surface Albedo', 'Wind Speed', 'Relative Humidity', 'Temperature',
   'Pressure', 'Cloud Type', 'Fill Flag', 'Wind Direction',
   'Precipitable Water']]
sunlight_data = dup_clt_data[['DHI', 'DNI', 'Clearsky DNI', 'Clearsky DHI', 'Clearsky GHI', 'GHI',
   'Solar Zenith Angle', 'Global Horizontal UV Irradiance (280-400nm)',
   'Global Horizontal UV Irradiance (295-385nm)']]
time_data = dup_clt_data[['Year', 'Month', 'Day', 'Hour', 'Minute','LocalTime', 'TIME',
   'DATE']]

# shift weather and solar data and combine back
weather_data = weather_data.shift(periods=8)
time_data = time_data.join(weather_data)
temp = time_data.join(sunlight_data).dropna()
temp.to_csv('Charlotte_weather_4h_timeshift.csv', index=False)  

In [94]:
temp[['Solar Zenith Angle','TIME','DATE']]

,Solar Zenith Angle,TIME,DATE
8,132.46,04:00:00,2006-01-01
9,126.34,04:30:00,2006-01-01
10,120.25,05:00:00,2006-01-01
11,114.21,05:30:00,2006-01-01
12,108.27,06:00:00,2006-01-01
...,...,...,...
17515,139.86,21:30:00,2006-12-31
17516,145.93,22:00:00,2006-12-31
17517,151.86,22:30:00,2006-12-31
17518,157.53,23:00:00,2006-12-31


In [95]:
irradiance_data_charlotte[['Solar Zenith Angle','TIME','DATE']]

,Solar Zenith Angle,TIME,DATE
0,166.47,00:00:00,2006-01-01
1,167.78,00:30:00,2006-01-01
2,165.82,01:00:00,2006-01-01
3,161.63,01:30:00,2006-01-01
4,156.37,02:00:00,2006-01-01
...,...,...,...
17515,139.86,21:30:00,2006-12-31
17516,145.93,22:00:00,2006-12-31
17517,151.86,22:30:00,2006-12-31
17518,157.53,23:00:00,2006-12-31


In [96]:
actual_solar_raleigh = pd.read_csv('Raleigh/6ef6c64fbb3bfd36bcb328cc956df78d/Actual_35.85_-78.65_2006_DPV_37MW_5_Min.csv')

actual_solar_raleigh['LocalTime'] = pd.to_datetime(actual_solar_raleigh['LocalTime'])
# adding separate time and date columns
actual_solar_raleigh["Year"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.year
actual_solar_raleigh['Day'] = pd.to_datetime(actual_solar_raleigh['LocalTime']).dt.day
actual_solar_raleigh['Month'] = pd.to_datetime(actual_solar_raleigh['LocalTime']).dt.month
actual_solar_raleigh["TIME"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.time
actual_solar_raleigh["DATE"] = pd.to_datetime(actual_solar_raleigh["LocalTime"]).dt.date
# add hours and minutes for ml models
actual_solar_raleigh['Hour'] = pd.to_datetime(actual_solar_raleigh['TIME'],format='%H:%M:%S').dt.hour
actual_solar_raleigh['Minute'] = pd.to_datetime(actual_solar_raleigh['TIME'],format='%H:%M:%S').dt.minute
actual_solar_raleigh['TOTAL MINUTES PASS'] = actual_solar_raleigh['Minute'] + actual_solar_raleigh['Hour']*60
actual_solar_raleigh.to_csv('Raleigh_Solar_Power.csv', index=False) 